In [70]:
# importing the dependencies
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import keras
from keras import backend as k
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

In [71]:
# a function to resize the image into appropriate dimensions
def resize(img):
    img = cv2.resize(img,(20,20))
    return img
    

In [72]:
X_train = []
y_train = []

# to get the name of the folder
for name_folder in os.listdir("extracted_letter_images") :

    name = 'extracted_letter_images/' + name_folder
    for f in listdir(name):
        # name of the folder is the name of the output
        y_train.append(np.asarray(name_folder))
        
        # constructing full path to the image
        name = 'extracted_letter_images/' + name_folder + '/' + f
        
        # reading the image
        image = cv2.imread(name,0)/255
        
        # appending to form the image list
        image = np.asarray(image)
        image = resize(image)
        X_train.append([image])

In [73]:
# converting the lsit into an numoy array so that it can be fed into neural network
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [74]:
X_train = np.reshape(X_train, [-1,20,20,1])

In [75]:
X_train.shape

(38744, 20, 20, 1)

In [76]:
# one hot encoding the output labels
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

In [77]:
# defining the architecture of the model
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(64, (3, 3), padding="same", input_shape=(20,20,1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# third convolutional layer with max pooling
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(100, activation="relu"))

# Output layer with 32 nodes (one for each possible letter/number we predict)
model.add(Dense(32, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 20, 20, 64)        640       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 10, 10, 128)       73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 5, 5, 512)         590336    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2048)              0         
__________

In [79]:
# splitting the samples into training and testing sets so that we cross validate our results
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,test_size = .2)

In [80]:
# shape of the training output
y_train.shape

(30995, 32)

In [81]:
# fitting the model
model.fit(X_train, y_train,
          validation_data=[X_test,y_test],
          batch_size=2000, 
          epochs=10, 
          verbose=1)


Train on 30995 samples, validate on 7749 samples
Epoch 1/10
30995/30995 [==============================] - 261s 8ms/step - loss: 3.3081 - acc: 0.1365 - val_loss: 2.7608 - val_acc: 0.4038
Epoch 2/10
30995/30995 [==============================] - 255s 8ms/step - loss: 1.6283 - acc: 0.6737 - val_loss: 0.6698 - val_acc: 0.8046
Epoch 3/10
30995/30995 [==============================] - 264s 9ms/step - loss: 0.3820 - acc: 0.9124 - val_loss: 0.1801 - val_acc: 0.9689
Epoch 4/10
30995/30995 [==============================] - 259s 8ms/step - loss: 0.1325 - acc: 0.9765 - val_loss: 0.0841 - val_acc: 0.9857
Epoch 5/10
30995/30995 [==============================] - 259s 8ms/step - loss: 0.0741 - acc: 0.9881 - val_loss: 0.0589 - val_acc: 0.9907
Epoch 6/10
30995/30995 [==============================] - 262s 8ms/step - loss: 0.0550 - acc: 0.9912 - val_loss: 0.0445 - val_acc: 0.9923
Epoch 7/10
30995/30995 [==============================] - 276s 9ms/step - loss: 0.0434 - acc: 0.9924 - val_loss: 0.0363 - v

from the above training and testing results we can say that our model performs very well on the data set

In [83]:
#saving the model 
model.save('models/model.h5')

In [84]:
np.save('models/xtrain',X_train,allow_pickle=True)